In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import cartopy as cp
from matplotlib.patches import FancyArrowPatch
from scipy.ndimage import map_coordinates
import metpy.calc as mpcalc
from geographiclib.geodesic import Geodesic
from utils_datetime import *
from utils_filter import *

In [4]:
displacements = xr.open_dataset('data/displacement/displacements.nc')
pph = xr.open_dataset('data/pph/labelled_pph.nc')
grid_outlooks = xr.open_dataset('data/outlooks/grid_outlooks.nc')
displacements

<xarray.Dataset>
Dimensions:    (y: 65, x: 93, time: 16436, hazard: 4)
Coordinates:
  * time       (time) object '197901010000' '197901020000' ... '202312310000'
  * x          (x) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 88.0 89.0 90.0 91.0 92.0
  * y          (y) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 60.0 61.0 62.0 63.0 64.0
  * hazard     (hazard) object 'Wind' 'Hail' 'Tornado' 'All Hazard'
Data variables:
    lat        (y, x) float64 ...
    lon        (y, x) float64 ...
    x_flow     (time, y, x, hazard) float64 ...
    y_flow     (time, y, x, hazard) float64 ...
    end_lon    (time, y, x, hazard) float64 ...
    end_lat    (time, y, x, hazard) float64 ...
    e_flow     (time, y, x, hazard) float64 ...
    n_flow     (time, y, x, hazard) float64 ...
    e_shift    (time, hazard) float64 ...
    n_shift    (time, hazard) float64 ...
    total_div  (time, hazard) float64 ...
Attributes:
    description:  Displacements from gridded day 1 probabilistic outlooks to ...
    grid:         80-km NCEP 211

In [5]:
grid_outlooks

<xarray.Dataset>
Dimensions:  (time: 16436, x: 93, y: 65, outlook: 13)
Coordinates:
  * time     (time) object '197901010000' '197901020000' ... '202312310000'
  * x        (x) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 88.0 89.0 90.0 91.0 92.0
  * y        (y) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 60.0 61.0 62.0 63.0 64.0
  * outlook  (outlook) object 'Day 3' 'Day 2 7' ... 'Day 1 Tornado' 'Day 1'
Data variables:
    lat      (y, x) float64 ...
    lon      (y, x) float64 ...
    prob     (time, y, x, outlook) float64 ...
Attributes:
    description:  outlook as a percentage as a function of date, lat/lon, and...
    grid:         80-km NCEP 211

In [ ]:
# creates new displacements dataset by keeping data (get rid of shifts and div) 
# but changing x and y coords so that (0, 0) is at max or weighted center
# then calculate things like shifts of quadrents

# note: grids will not quite be aligned when each is centered over a different point, but probably close enough